In [4]:
from dotenv import load_dotenv
import os
from groq import Groq
import pandas as pd
import oracledb
import re

In [2]:
file_path = "financial_statements_aapl.xlsx"  # Update with your file name
df = pd.read_excel(file_path)

# Display the first few rows
print(df.head())

  Fiscal Quarter  Revenue Revenue Growth (YoY)  Cost of Revenue  Gross Profit  \
0        Q4 2024    94930                6.07%            51051         43879   
1        Q3 2024    85777                4.87%            46099         39678   
2        Q2 2024    90753               -4.31%            48482         42271   
3        Q1 2024   119575                2.07%            64720         54855   
4        Q4 2023    89498               -0.72%            49071         40427   

   Selling, General & Admin  Research & Development  Operating Expenses  \
0                      6523                    7765               14288   
1                      6320                    8006               14326   
2                      6468                    7903               14371   
3                      6786                    7696               14482   
4                      6151                    7307               13458   

   Operating Income Interest Expense  ... Operating Margin Pro

In [55]:
import pandas as pd
import oracledb
import re
import numpy as np
import locale

# Set locale for correct number parsing
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Load Excel file
file_path = "financial_statements_aapl.xlsx"
xls = pd.ExcelFile(file_path)

# Function to clean column names
def clean_column_names(columns):
    cleaned_columns = []
    for col in columns:
        col = col.replace(",", " ")  # Replace ',' with space
        col = col.replace("&", " and ")  # Replace '&' with 'and'
        col = col.replace("(", "_").replace(")", "_")  # Replace '()' with '_'
        col = col.replace("-", " ")  # Replace '-' with space
        col = re.sub(r"\s+", " ", col).strip()  # Remove extra spaces
        cleaned_columns.append(col)
    return cleaned_columns

# Function to convert string numbers & percentages to floats
def convert_to_float(value):
    """Converts numbers and percentages to float while handling missing values."""
    if pd.isna(value) or value in ["", "-"]:
        return np.nan  # Keep missing values as NaN

    value = str(value).strip()

    # Convert percentage to decimal
    if value.endswith("%"):
        try:
            return locale.atof(value[:-1]) / 100  # Convert "55%" → 0.55
        except ValueError:
            return np.nan

    # Convert standard numbers
    try:
        return locale.atof(value)  # Convert "55" → 55.0
    except ValueError:
        return np.nan  # Default to NaN for non-numeric values

# Function to detect column types
def detect_column_type(column_name, series):
    """Assigns FLOAT to all columns except 'Fiscal Quarter' and 'Fiscal Year'."""
    if "fiscal year" in column_name.lower() or "fiscal quarter" in column_name.lower():
        return "VARCHAR2(255)"  # Keep Fiscal Year & Fiscal Quarter as text

    return "FLOAT"  # Everything else is FLOAT

# Function to clean the data and apply transformations
def clean_data(df):
    """Clean data by handling NaNs, converting percentages, and ensuring correct types."""

    # Replace empty strings and dashes with NaN
    df.replace(["-", ""], np.nan, inplace=True)

    # Convert all columns (except Fiscal Year & Fiscal Quarter) to FLOAT
    for col in df.columns:
        col_type = detect_column_type(col, df[col])

        if col_type == "FLOAT":
            df[col] = df[col].apply(convert_to_float)  # Convert numbers & percentages
        
    return df

# Function to upload cleaned data to Oracle
def upload_to_oracle(sheet_name, df):
    # Oracle connection
    user = "ADMIN"
    password = "Passwordtestdb@1"
    dsn = "testdb_medium"
    wallet_location = r"/Users/vexy/Documents/Wallet_testdb/"

    conn = oracledb.connect(
        user=user,
        password=password,
        dsn=dsn,
        config_dir=wallet_location,
        wallet_location=wallet_location,
        wallet_password=password
    )
    
    cursor = conn.cursor()

    # Format table name
    table_name = re.sub(r"\W+", "_", sheet_name).upper()

    # Drop table if exists
    drop_table_query = f"BEGIN EXECUTE IMMEDIATE 'DROP TABLE {table_name}'; EXCEPTION WHEN OTHERS THEN NULL; END;"
    cursor.execute(drop_table_query)

    # Determine correct SQL data types for each column
    column_definitions = []
    for col in df.columns:
        col_type = detect_column_type(col, df[col])
        column_definitions.append(f'"{col}" {col_type}')

    # Create table
    create_table_query = f"CREATE TABLE {table_name} ({', '.join(column_definitions)})"
    cursor.execute(create_table_query)
    print(f"Table {table_name} created in Oracle.")

    # Prepare INSERT query
    column_names = ", ".join([f'"{col}"' for col in df.columns])
    placeholders = ", ".join([f":{i+1}" for i in range(len(df.columns))])  # Oracle placeholders
    query = f'INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})'

    # Convert DataFrame rows to tuples, replacing NaN with None (NULL in Oracle)
    data_tuples = [tuple(None if pd.isna(x) else x for x in row) for row in df.to_numpy()]

    # Insert data
    cursor.executemany(query, data_tuples)
    conn.commit()

    print(f"Data from '{sheet_name}' uploaded to table '{table_name}' successfully.")

    # Close connection
    cursor.close()
    conn.close()

# Process all sheets
for sheet in xls.sheet_names:
    df = xls.parse(sheet, dtype=str)  # Ensure all columns are loaded as strings initially
    df.columns = clean_column_names(df.columns)  # Clean column names
    df = clean_data(df)  # Clean and transform data
    upload_to_oracle(sheet, df)  # Upload to Oracle

print("All sheets cleaned and uploaded successfully!")


/var/folders/cl/79fsf7m90k3bx5hjvvt29pzr0000gn/T/ipykernel_39890/3566351203.py:60: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(["-", ""], np.nan, inplace=True)


Table INCOME_QUARTERLY created in Oracle.
Data from 'income quarterly' uploaded to table 'INCOME_QUARTERLY' successfully.


/var/folders/cl/79fsf7m90k3bx5hjvvt29pzr0000gn/T/ipykernel_39890/3566351203.py:60: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(["-", ""], np.nan, inplace=True)


Table BALANCE_SHEET_QUARTERLY created in Oracle.
Data from 'balance sheet quarterly' uploaded to table 'BALANCE_SHEET_QUARTERLY' successfully.


/var/folders/cl/79fsf7m90k3bx5hjvvt29pzr0000gn/T/ipykernel_39890/3566351203.py:60: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(["-", ""], np.nan, inplace=True)


Table CASH_FLOW_QUARTERLY created in Oracle.
Data from 'cash flow quarterly' uploaded to table 'CASH_FLOW_QUARTERLY' successfully.
Table RATIO_QUARTERLY created in Oracle.
Data from 'ratio quarterly' uploaded to table 'RATIO_QUARTERLY' successfully.
All sheets cleaned and uploaded successfully!


In [57]:
user = "ADMIN"
password = "Passwordtestdb@1"
dsn = "testdb_medium"
wallet_location = r"/Users/vexy/Documents/Wallet_testdb/"

# Establish database connection
conn = oracledb.connect(
    user=user,
    password=password,
    dsn=dsn,
    config_dir=wallet_location,
    wallet_location=wallet_location,
    wallet_password=password
)
    
cursor = conn.cursor()


# Query to list all tables in the user's schema
query = "SELECT table_name FROM user_tables"
cursor.execute(query)

# Fetch and print all table names
tables = cursor.fetchall()

print("Tables in Oracle Database:")
for table in tables:
    print(table[0])  # Print table name only

# Close connection
cursor.close()
conn.close()

Tables in Oracle Database:
SHEET
INCOME_QUARTERLY
BALANCE_SHEET_QUARTERLY
CASH_FLOW_QUARTERLY
RATIO_QUARTERLY
DBTOOLS$EXECUTION_HISTORY


In [58]:
# Execute Query

user = "ADMIN"
password = "Passwordtestdb@1"
dsn = "testdb_medium"
wallet_location = r"/Users/vexy/Documents/Wallet_testdb/"

# Establish database connection
conn = oracledb.connect(
    user=user,
    password=password,
    dsn=dsn,
    config_dir=wallet_location,
    wallet_location=wallet_location,
    wallet_password=password
)
    
cursor = conn.cursor()
query = "SELECT * FROM INCOME_QUARTERLY"
cursor.execute(query)

# Fetch Results
rows = cursor.fetchall()
for row in rows:
    print(row)
# Close Connection
cursor.close()
conn.close()

('Q4 2024', 94930.0, 0.060700000000000004, 51051.0, 43879.0, 6523.0, 7765.0, 14288.0, 29591.0, None, None, None, 19.0, 29610.0, None, 29610.0, 14874.0, 14736.0, 14736.0, -0.35810000000000003, 15172.0, 15243.0, -0.0274, 0.97, 0.96, -0.3407, 23903.0, 1.57, 0.25, 0.0417, 0.4622, 0.31170000000000003, 0.1552, 0.2518, 32502.0, 0.34240000000000004, 2911.0, 29591.0, 0.31170000000000003, 0.5023, 94930.0)
('Q3 2024', 85777.0, 0.0487, 46099.0, 39678.0, 6320.0, 8006.0, 14326.0, 25352.0, None, None, None, 142.0, 25494.0, None, 25494.0, 4046.0, 21448.0, 21448.0, 0.0788, 15288.0, 15348.0, -0.0271, 1.4, 1.4, 0.11109999999999999, 26707.0, 1.74, 0.25, 0.0417, 0.46259999999999996, 0.2956, 0.25, 0.3114, 28202.0, 0.32880000000000004, 2850.0, 25352.0, 0.2956, 0.15869999999999998, 85777.0)
('Q2 2024', 90753.0, -0.0431, 48482.0, 42271.0, 6468.0, 7903.0, 14371.0, 27900.0, None, None, None, 158.0, 28058.0, None, 28058.0, 4422.0, 23636.0, 23636.0, -0.0217, 15406.0, 15465.0, -0.0241, 1.53, 1.53, 0.0066, 20694.0, 

In [59]:
# Get all table names
user = "ADMIN"
password = "Passwordtestdb@1"
dsn = "testdb_medium"
wallet_location = r"/Users/vexy/Documents/Wallet_testdb/"

# Establish database connection
conn = oracledb.connect(
    user=user,
    password=password,
    dsn=dsn,
    config_dir=wallet_location,
    wallet_location=wallet_location,
    wallet_password=password
)
    
cursor = conn.cursor()
cursor.execute("SELECT table_name FROM user_tables")
tables = cursor.fetchall()

print("\nDDL Statements for All Tables:\n")

for table in tables:
    table_name = table[0]

    # Get DDL for the table
    ddl_query = f"SELECT DBMS_METADATA.GET_DDL('TABLE', '{table_name}') FROM DUAL"
    cursor.execute(ddl_query)
    ddl_statement = cursor.fetchone()[0].read()

    print(f"\nDDL for Table: {table_name}\n" + "-" * 40)
    print(ddl_statement)
    print("\n" + "-" * 80)

# Close connection
cursor.close()
conn.close()


DDL Statements for All Tables:


DDL for Table: SHEET
----------------------------------------

  CREATE TABLE "ADMIN"."SHEET" 
   (	"Interest Expense" VARCHAR2(255) COLLATE "USING_NLS_COMP"
   )  DEFAULT COLLATION "USING_NLS_COMP" SEGMENT CREATION IMMEDIATE 
  PCTFREE 10 PCTUSED 40 INITRANS 10 MAXTRANS 255 
 COLUMN STORE COMPRESS FOR QUERY HIGH ROW LEVEL LOCKING LOGGING
  STORAGE(INITIAL 65536 NEXT 1048576 MINEXTENTS 1 MAXEXTENTS 2147483645
  PCTINCREASE 0 FREELISTS 1 FREELIST GROUPS 1
  BUFFER_POOL DEFAULT FLASH_CACHE DEFAULT CELL_FLASH_CACHE DEFAULT)
  TABLESPACE "DATA" 

--------------------------------------------------------------------------------

DDL for Table: INCOME_QUARTERLY
----------------------------------------

  CREATE TABLE "ADMIN"."INCOME_QUARTERLY" 
   (	"Fiscal Quarter" VARCHAR2(255) COLLATE "USING_NLS_COMP", 
	"Revenue" FLOAT(126), 
	"Revenue Growth _YoY_" FLOAT(126), 
	"Cost of Revenue" FLOAT(126), 
	"Gross Profit" FLOAT(126), 
	"Selling General and Admin" FLOA

In [60]:
user = "ADMIN"
password = "Passwordtestdb@1"
dsn = "testdb_medium"
wallet_location = r"/Users/vexy/Documents/Wallet_testdb/"

# Establish database connection
conn = oracledb.connect(
    user=user,
    password=password,
    dsn=dsn,
    config_dir=wallet_location,
    wallet_location=wallet_location,
    wallet_password=password
)
    
cursor = conn.cursor()

with open("oracle_table_schemas.sql", "w") as f:
    for table in tables:
        table_name = table[0]

        ddl_query = f"SELECT DBMS_METADATA.GET_DDL('TABLE', '{table_name}') FROM DUAL"
        cursor.execute(ddl_query)
        ddl_statement = cursor.fetchone()[0].read()

        f.write(f"\n-- DDL for {table_name}\n")
        f.write(ddl_statement + "\n\n")

print("DDL statements saved to oracle_table_schemas.sql")

# Close connection
cursor.close()
conn.close()

DDL statements saved to oracle_table_schemas.sql


In [61]:
import re

# File paths
input_file = "oracle_table_schemas.sql"
output_file = "cleaned_oracle_table_schemas.sql"

# Read the original SQL file
with open(input_file, "r", encoding="utf-8") as file:
    sql_contents = file.read()

# Remove "COLLATE USING_NLS_COMP"
cleaned_sql = re.sub(r'\s*COLLATE\s+"USING_NLS_COMP"', '', sql_contents)

# Save the cleaned SQL to a new file
with open(output_file, "w", encoding="utf-8") as file:
    file.write(cleaned_sql)

print(f"Cleaned SQL saved to: {output_file}")


Cleaned SQL saved to: cleaned_oracle_table_schemas.sql


In [2]:
load_dotenv()  # Load environment variables from .env
api_key = os.getenv("GROQ_API_KEY")

client = Groq(api_key=api_key)

In [3]:
completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {
            "role": "user",
            "content": "Based on the DDL below, answer the query with an SQL query as the only output\nddl = \"\"\"CREATE TABLE financial_metrics (\n    metric_id SERIAL PRIMARY KEY,  -- Unique identifier for each metric\n    metric_name TEXT NOT NULL,     -- The name of the financial metric\n    year_2007 VARCHAR(50),         -- Value for the year 2007\n    year_2006 VARCHAR(50),         -- Value for the year 2006\n    year_2005 VARCHAR(50)          -- Value for the year 2005\n);\"\"\"\n\n##natural language query\nquery = \"By how much did the weighted average exercise price per share increase from 2005 to 2007?\"\n\n"
        },
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

```sql
SELECT SUM(CAST(REPLACE(year_2007, ',', '') AS FLOAT) - CAST(REPLACE(year_2005, ',', '') AS FLOAT)) as increase
FROM financial_metrics
WHERE metric_name = 'Weighted Average Exercise Price per Share';
```

This query assumes the amount values in years 2005 and 2007 are in comma-separated format. The `REPLACE` function is used to remove any commas, then the result is cast to a float to be used in the arithmetic operation.

In [39]:
a= "56"
a= float(a)
print(type(a))
print(a)

<class 'float'>
56.0


In [89]:
# Read the cleaned Oracle table schemas
ddl_file_path = "cleaned_oracle_table_schemas.sql"

with open(ddl_file_path, "r", encoding="utf-8") as file:
    ddl_content = file.read().strip()  # Read and remove unnecessary whitespace

# Define the natural language query
nl_query = "What was the gross profit in q3 2024?"

load_dotenv()  # Load environment variables from .env
api_key = os.getenv("GROQ_API_KEY")

client = Groq(api_key=api_key)

# Format the query dynamically
prompt = f"""Based on the DDL below, answer the question with an SQL query that starts with "SQL:".For example SQL:SELECT * FROM TABLE;. Do not use any otehr prefix for the SQL query.Any extra information should be prefixed by 'NOTE:'.For example NOTE: The data requires etc. If you do not have appropriate information, reply with "I am sorry, I am having trouble finding the right information." 

ddl = \"\"\"{ddl_content}\"\"\"

## Natural Language Query
query = \"{nl_query}\"
"""

# Call the LLM API
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": prompt
        },
    ],
    temperature=.7,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

# Collect the streamed output into a single string
llm_output = ""
for chunk in completion:
    if hasattr(chunk.choices[0].delta, "content"):  # Ensure content exists
        llm_output += chunk.choices[0].delta.content or ""

# Strip any unnecessary whitespace
llm_output = llm_output.strip()

# Use regex to extract SQL and Notes
sql_match = re.search(r"SQL:\s*(.*?)(?:\nNOTES:|$)", llm_output, re.DOTALL)
notes_match = re.search(r"NOTES:\s*(.*)", llm_output, re.DOTALL)

# Store the extracted values
query = sql_match.group(1).strip() if sql_match else None  # Extract SQL part
extra = notes_match.group(1).strip() if notes_match else None  # Extract Notes
query = query.rstrip(";")  # Removes trailing semicolon if present

# Debugging: Print to verify extraction (Remove later)
print("Query:", query)
print("Extra:", extra)



Query: SELECT "Gross Profit" FROM "ADMIN"."INCOME_QUARTERLY" WHERE "Fiscal Quarter" = 'Q3 2024'
Extra: None


In [90]:
print("Query:", query)

Query: SELECT "Gross Profit" FROM "ADMIN"."INCOME_QUARTERLY" WHERE "Fiscal Quarter" = 'Q3 2024'


In [91]:


# Establish database connection
conn = oracledb.connect(
    user=user,
    password=password,
    dsn=dsn,
    config_dir=wallet_location,
    wallet_location=wallet_location,
    wallet_password=password
)
    
cursor = conn.cursor()
try:
    # Execute the SQL query
    cursor.execute(query)

    # Fetch results
    results = cursor.fetchall()

    # Print results
    print("Query Results:")
    for row in results:
        print(row)

except oracledb.DatabaseError as e:
    print("Error executing query:", e)

finally:
    # Close cursor and connection
    cursor.close()
    conn.close()

Query Results:
(39678.0,)
